In [70]:
# Import necessary modules
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import pandas as pd
import numpy as np

In [71]:
# Create a sample dataframe (in practice, you would load your own)
data = {
    'feature1': [0.1, 0.2, 0.3, 0.9, 1.0, 1.1],
    'feature2': [1.2, 0.9, 1.1, 0.3, 0.2, 0.1],
    'label':    [0, 1, 2, 0, 1, 2]
}
df = pd.DataFrame(data)

In [72]:
# This custom Dataset class turns the DataFrame into something PyTorch can use
class MyDataset(Dataset):
    def __init__(self, dataframe):
        # What we're basing our prediction off of
        features = dataframe[['feature1', 'feature2']].to_numpy().astype(np.float32)
        # Our classifications
        labels = dataframe['label'].to_numpy().astype(np.int64)
        
        self.X = torch.from_numpy(features)
        self.y = torch.from_numpy(labels)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [73]:
# Define a very simple neural network model
class MultiClassClassifier(nn.Module):
    def __init__(self, input_size, num_classes):
        super().__init__()
        # 8 is the number of neurons
        self.hidden = nn.Linear(input_size, 8)
        # ReLU, short for Rectified Linear Unit, just does max(0, x)
        # It helps the network learn non-linear patterns
        self.relu = nn.ReLU()
        self.output = nn.Linear(8, num_classes)

    def forward(self, x):
        x = self.hidden(x)
        x = self.relu(x)
        x = self.output(x)
        return x

In [74]:
# Prepare data
dataset = MyDataset(df)
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

# Initialize model, loss, and optimizer
model = MultiClassClassifier(input_size=2, num_classes=3)
# CrossEntropyLoss applies a softmax under the hood to turn logits (raw scores) into probabilities between 0 and 1.
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [75]:
# Training loop
for epoch in range(10):
    for inputs, labels in dataloader:
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

Epoch 1, Loss: 1.2499
Epoch 2, Loss: 1.3106
Epoch 3, Loss: 1.2231
Epoch 4, Loss: 1.2403
Epoch 5, Loss: 0.9271
Epoch 6, Loss: 1.2480
Epoch 7, Loss: 1.0940
Epoch 8, Loss: 1.1814
Epoch 9, Loss: 1.0747
Epoch 10, Loss: 1.1700


In [76]:
# Prediction
with torch.no_grad():
    sample = torch.tensor([[0.2, 1.0]], dtype=torch.float32)
    # The raw "scores" for each classification are called logits
    logits = model(sample)
    # Argmax finds the index of the highest score
    predicted_class = torch.argmax(logits, dim=1).item()
    print(f"Predicted Classification: {predicted_class}")

Predicted Classification: 0
